<a href="https://colab.research.google.com/github/Kashaf22/ArcGIS_Scripts/blob/main/InUse_LSTM_Model3_Updated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import json
import numpy as np
from keras.models import Model
from keras.layers import LSTM, Dense, Dropout, Input, Concatenate
from keras.regularizers import l2

# Load the data from the JSON file
with open('/content/sample_data/realistic_trajectory_data-24.json', 'r') as file:
    data = json.load(file)

# Extract the trajectories and initial accelerations from the data
trajectories = []
initial_accs = []
for point in data:
    trajectories.append([point['x'], point['y'], point['z']])
    initial_accs.append([point['ax'], point['ay'], point['az']])

# Define the number of timesteps and features
num_timesteps = 13  # Number of previous trajectory points to consider
num_pos_features = 3  # x, y, z coordinates

# Prepare the training data
X_train_pos = []
X_train_acc = []
y_train = []
for i in range(len(trajectories) - num_timesteps):
    X_train_pos.append(trajectories[i:i+num_timesteps])
    X_train_acc.append(initial_accs[i])
    y_train.append(trajectories[i + num_timesteps])

# Convert to numpy arrays for training
X_train_pos = np.array(X_train_pos)
X_train_acc = np.array(X_train_acc)
y_train = np.array(y_train)

# Define the LSTM model
input_pos = Input(shape=(num_timesteps, num_pos_features))
input_acc = Input(shape=(3,))

x = LSTM(128, input_shape=(num_timesteps, num_pos_features), return_sequences=True, kernel_regularizer=l2(0.01))(input_pos)
x = Dropout(0.2)(x)
x = LSTM(128, return_sequences=True, kernel_regularizer=l2(0.01))(x)
x = Dropout(0.2)(x)
x = LSTM(64, kernel_regularizer=l2(0.01))(x)
combined = Concatenate(axis=-1)([x, input_acc])
output = Dense(num_pos_features, kernel_regularizer=l2(0.01))(combined)

model = Model(inputs=[input_pos, input_acc], outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit([X_train_pos, X_train_acc], y_train, epochs=200, batch_size=64, validation_split=0.2)

# Save the trained model
model.save('trajectory_model.h5')

Epoch 1/200
19/19 [==============================] - 11s 258ms/step - loss: 66266.1875 - val_loss: 68390.6797
Epoch 2/200
19/19 [==============================] - 2s 87ms/step - loss: 47550.1602 - val_loss: 50576.7188
Epoch 3/200
19/19 [==============================] - 2s 88ms/step - loss: 34633.7461 - val_loss: 38285.8438
Epoch 4/200
19/19 [==============================] - 2s 88ms/step - loss: 25942.5996 - val_loss: 29330.6113
Epoch 5/200
19/19 [==============================] - 2s 86ms/step - loss: 19653.9219 - val_loss: 22997.4336
Epoch 6/200
19/19 [==============================] - 2s 87ms/step - loss: 15210.1729 - val_loss: 18261.3281
Epoch 7/200
19/19 [==============================] - 3s 141ms/step - loss: 12002.9873 - val_loss: 14741.5391
Epoch 8/200
19/19 [==============================] - 2s 83ms/step - loss: 9690.0645 - val_loss: 12178.2500
Epoch 9/200
19/19 [==============================] - 2s 82ms/step - loss: 8020.9883 - val_loss: 10186.5361
Epoch 10/200
19/19 [=======

In [19]:
import numpy as np
from keras.models import load_model

# Load the trained model
model = load_model('trajectory_model.h5')

# Initial position and acceleration
initial_pos = np.array([3.075831475154961, 4.206734691502548, 2.950718916245421])
initial_acc = np.array([1046.9889099743164, 1431.9394949898867, 1057.642122414968])

# Prepare the initial sequence with the single initial position and constant acceleration
# Assuming the model was trained with sequences where acceleration is constant and only the positions vary
input_seq_pos = [initial_pos] * num_timesteps  # Replicate the initial position to fill the sequence
input_seq_acc = initial_acc.reshape(1, 3)  # Keep the acceleration constant

predicted_trajectory = []

# Predict the next 5 positions
for _ in range(8):
    X_test_pos = np.array(input_seq_pos[-num_timesteps:]).reshape(1, num_timesteps, num_pos_features)  # Use the most recent positions
    next_pos = model.predict([X_test_pos, input_seq_acc])[0]  # Predict the next position

    # Update the input sequence and store the prediction
    input_seq_pos.append(next_pos)  # Add the predicted position to the input sequence
    predicted_trajectory.append(next_pos)  # Store the prediction

print("Initial Position:", initial_pos.tolist())
print("Initial Acceleration:", initial_acc.tolist())
print("Predicted Trajectory:")
for pos in predicted_trajectory:
    print(pos.tolist())


1/1 [==============================] - 0s 31ms/step
Initial Position: [3.075831475154961, 4.206734691502548, 2.950718916245421]
Initial Acceleration: [1046.9889099743164, 1431.9394949898867, 1057.642122414968]
Predicted Trajectory:
[22.45094108581543, 22.698659896850586, 5.930832862854004]
[22.213090896606445, 22.464784622192383, 5.763583183288574]
[21.93588638305664, 22.148523330688477, 5.5632100105285645]
[21.910865783691406, 21.918987274169922, 5.323182106018066]
[22.653284072875977, 22.06784439086914, 5.012879371643066]
[25.961118698120117, 23.92301368713379, 4.681106090545654]
[32.72053146362305, 28.896780014038086, 4.372860908508301]
[40.161495208740234, 35.70177459716797, 3.5946357250213623]


In [21]:
import json
import numpy as np
from keras.models import Model, load_model
from keras.layers import LSTM, Dense, Dropout, Input, Concatenate
from keras.regularizers import l2

# Load the trained model
model = load_model('trajectory_model.h5')

# Define the number of timesteps and features
num_timesteps = 13 # Number of previous trajectory points to consider
num_pos_features = 3  # x, y, z coordinates

# Test the model with a random initial position and acceleration
initial_pos = [3.075831475154961,4.206734691502548,2.950718916245421]
initial_acc = [1046.9889099743164,1431.9394949898867,1057.642122414968]
input_seq = [initial_pos]
predicted_trajectory = [initial_pos]

for _ in range(num_timesteps - 1):
    X_pos = np.array(input_seq[-num_timesteps:]) if len(input_seq) >= num_timesteps else np.zeros((num_timesteps, num_pos_features))
    X_test_pos = X_pos.reshape(1, num_timesteps, num_pos_features)
    X_test_acc = np.array(initial_acc).reshape(1, 3)
    next_pos = model.predict([X_test_pos, X_test_acc])
    input_seq.append(next_pos[0])
    predicted_trajectory.append(next_pos[0])

# Predict the next 5 positions
for _ in range(13):
    X_pos = np.array(input_seq[-num_timesteps:])
    X_test_pos = X_pos.reshape(1, num_timesteps, num_pos_features)
    X_test_acc = np.array(initial_acc).reshape(1, 3)
    next_pos = model.predict([X_test_pos, X_test_acc])
    input_seq.append(next_pos[0])
    predicted_trajectory.append(next_pos[0])

print("Initial Position: ", initial_pos)
print("Initial Acceleration: ", initial_acc)
print("Predicted Trajectory:")
for pos in predicted_trajectory:
    print(pos)

1/1 [==============================] - 0s 32ms/step
Initial Position:  [3.075831475154961, 4.206734691502548, 2.950718916245421]
Initial Acceleration:  [1046.9889099743164, 1431.9394949898867, 1057.642122414968]
Predicted Trajectory:
[3.075831475154961, 4.206734691502548, 2.950718916245421]
[34.862854  38.91199    3.6628804]
[34.862854  38.91199    3.6628804]
[34.862854  38.91199    3.6628804]
[34.862854  38.91199    3.6628804]
[34.862854  38.91199    3.6628804]
[34.862854  38.91199    3.6628804]
[34.862854  38.91199    3.6628804]
[34.862854  38.91199    3.6628804]
[34.862854  38.91199    3.6628804]
[34.862854  38.91199    3.6628804]
[34.862854  38.91199    3.6628804]
[34.862854  38.91199    3.6628804]
[ 2.0231404 -2.9881165  2.877731 ]
[-0.3631493 -1.290706   3.1133587]
[4.7582955 0.3470046 5.7173424]
[9.015968  2.4612586 8.86975  ]
[12.936681   3.8546302 11.181785 ]
[16.987772   4.8201756 13.211766 ]
[21.324932   5.8025746 14.928819 ]
[26.207426  7.222713 16.638325]
[30.071295  8.617

In [ ]:
from keras.models import load_model
import json
import numpy as np

# Load the previously trained model
model = load_model('trajectory_model.h5')

# Load new data from another JSON file
with open('/content/sample_data/realistic_trajectory_data-12.json', 'r') as file:
    new_data = json.load(file)

# Extract trajectories and initial accelerations from the new data
new_trajectories = []
new_initial_accs = []
for point in new_data:
    new_trajectories.append([point['x'], point['y'], point['z']])
    new_initial_accs.append([point['ax'], point['ay'], point['az']])

# Prepare the new training data
X_train_pos_new = []
X_train_acc_new = []
y_train_new = []
num_timesteps = 5  # This should match the value used during initial training
for i in range(len(new_trajectories) - num_timesteps):
    X_train_pos_new.append(new_trajectories[i:i+num_timesteps])
    X_train_acc_new.append(new_initial_accs[i])
    y_train_new.append(new_trajectories[i + num_timesteps])

# Convert to numpy arrays
X_train_pos_new = np.array(X_train_pos_new)
X_train_acc_new = np.array(X_train_acc_new)
y_train_new = np.array(y_train_new)

# Continue training the model on the new data
model.fit([X_train_pos_new, X_train_acc_new], y_train_new, epochs=200, batch_size=64, validation_split=0.2)

# Optionally save the updated model
model.save('updated_trajectory_model.h5')

In [ ]:
import json
import numpy as np
from keras.models import Model, load_model
from keras.layers import LSTM, Dense, Dropout, Input, Concatenate
from keras.regularizers import l2

# Load the trained model
model = load_model('updated_trajectory_model.h5')

# Define the number of timesteps and features
num_timesteps = 5  # Number of previous trajectory points to consider
num_pos_features = 3  # x, y, z coordinates

# Test the model with a random initial position and acceleration
initial_pos = [19.893560441900362,11.676007613329734,12.512333688615755]
initial_acc = [1612.2885586899642,946.2908131064622,1237.6817826067672]
input_seq = [initial_pos]
predicted_trajectory = [initial_pos]

for _ in range(num_timesteps - 1):
    X_pos = np.array(input_seq[-num_timesteps:]) if len(input_seq) >= num_timesteps else np.zeros((num_timesteps, num_pos_features))
    X_test_pos = X_pos.reshape(1, num_timesteps, num_pos_features)
    X_test_acc = np.array(initial_acc).reshape(1, 3)
    next_pos = model.predict([X_test_pos, X_test_acc])
    input_seq.append(next_pos[0])
    predicted_trajectory.append(next_pos[0])

# Predict the next 5 positions
for _ in range(5):
    X_pos = np.array(input_seq[-num_timesteps:])
    X_test_pos = X_pos.reshape(1, num_timesteps, num_pos_features)
    X_test_acc = np.array(initial_acc).reshape(1, 3)
    next_pos = model.predict([X_test_pos, X_test_acc])
    input_seq.append(next_pos[0])
    predicted_trajectory.append(next_pos[0])

print("Initial Position: ", initial_pos)
print("Initial Acceleration: ", initial_acc)
print("Predicted Trajectory:")
for pos in predicted_trajectory:
    print(pos)